In [ ]:
#networkx is used to create and process graphs.
#NetworkXError handles any graph-related errors
import networkx as nx
from networkx.exception import NetworkXError

def pagerank(G, alpha=0.85, personalization=None,  #G → input graph alpha → damping factor tol → stopping condition
             max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
             dangling=None):     #start → initial rank values dangling → nodes with no outgoing links

    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()   #PageRank works on directed graphs, so if it's undirected, convert it.
    else:
        D = G

    # Convert graph to stochastic form
    W = nx.stochastic_graph(D, weight=weight)  #Converts graph into probability transition format (each row sums to 1).
    N = W.number_of_nodes()    #Stores total number of nodes.

    # Initialize rank vector
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)  # if no initial values, assign equal rank 1/N to every node.
    else:
        s = float(sum(nstart.values()))
        x = {k: v / s for k, v in nstart.items()} #If initial values are provided, normalize them so total = 1.

    # Personalization vector
    if personalization is None:
        p = dict.fromkeys(W, 1.0 / N)  #If no personalization, use equal probability for all nodes
    else:
        missing = set(G) - set(personalization)  #Checks if any node is missing from personalization.
        if missing:
            raise NetworkXError(f"Personalization missing nodes: {missing}")
        s = float(sum(personalization.values()))    #Normalize personalization values.
        p = {k: v / s for k, v in personalization.items()}

    # Dangling nodes handling
    if dangling is None:
        dangling_weights = p   #If no dangling vector given, use personalization vector for it.
    else:
        missing = set(G) - set(dangling)   #Ensures all nodes exist in dangling dictionary.
        if missing:
            raise NetworkXError(f"Dangling dictionary missing nodes: {missing}")
        s = float(sum(dangling.values()))
        dangling_weights = {k: v / s for k, v in dangling.items()}  
 
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    #Finds nodes that have zero outgoing links.

    # Power iteration
    for _ in range(max_iter):
        xlast = x.copy()     #Save previous rank values for convergence check.
        x = dict.fromkeys(xlast.keys(), 0)     #Reset rank for new iteration.
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)  #Total rank lost due to dangling nodes, redistributed later.
 
        for n in x:
            for nbr in W[n]:                  # Distribute rank from node n to its neighbors nbr.
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

        # Check convergence (L1 norm)
        err = sum(abs(x[n] - xlast[n]) for n in x)  #Compute difference between new and old ranks.
        if err < N * tol:
            return x            #If difference is very small → stop early and return result.

    raise NetworkXError(f"PageRank failed to converge in {max_iter} iterations.")


In [ ]:
import networkx as nx

# In PageRank, damping factor (α) represents the probability that a user will:
# continue clicking the next link → α probability
#randomly jump to any other page → (1 − α) probability

G = nx.barabasi_albert_graph(60, 41)
#Creates a graph with 60 nodes, each connecting to 41 others.
pr = pagerank(G, alpha=0.4)    #Runs PageRank with damping factor 0.4 and prints scores.

print(pr)


{0: 0.02805009581317142, 1: 0.012149551779168336, 2: 0.012577178556923042, 3: 0.013368320767324294, 4: 0.01235902999867519, 5: 0.012760429829180102, 6: 0.013166434313000996, 7: 0.013370931927890796, 8: 0.013162439327920817, 9: 0.01297475828042739, 10: 0.013377296141376987, 11: 0.012758619341896413, 12: 0.012963140909757203, 13: 0.012790074184971034, 14: 0.01296667038759297, 15: 0.012181283050371071, 16: 0.012559017759709828, 17: 0.013570227254980078, 18: 0.012574796322044536, 19: 0.012365437691344015, 20: 0.013569631650820396, 21: 0.01295552964699519, 22: 0.01315110285730655, 23: 0.013568074677829023, 24: 0.01256989160799764, 25: 0.013357591275027697, 26: 0.013569210387615917, 27: 0.01316864141035763, 28: 0.01257419226084101, 29: 0.012756160509648055, 30: 0.012963024587279057, 31: 0.013162439327920817, 32: 0.012973829944259026, 33: 0.01295589739101076, 34: 0.013357591275027697, 35: 0.01254614051508213, 36: 0.012958335403099566, 37: 0.012386081360350758, 38: 0.013168917400840523, 39: 0.